In [1]:
from days.w2d4.attention import *
import torch as t
from torch import einsum
from torch.nn import Module
import einops
from typing import Callable
from torchtyping import patch_typeguard, TensorType
import numpy as np
import days.w2d1.bert_tests as tests
import torchtext
import gin
import transformers
from bert_finetuning import *
from torch.profiler import profile 

In [2]:
max_seq_len = 256
batch_size = 4
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
data_train, data_test = torchtext.datasets.IMDB(root=".data", split = ("train", "test"))
train_batches = extract_data(data_train, max_seq_len, batch_size, tokenizer)
test_batches = extract_data(data_test, max_seq_len, batch_size, tokenizer)

In [4]:
class_bert = setup_bert()
class_bert.train()
class_bert.cuda();

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokens, labels = next(train_batches)
tokens_cuda = tokens.input_ids.cuda()

with profile(with_stack=True, profile_memory=True) as prof:
        outputs = class_bert(tokens_cuda)[1]

In [6]:
print(prof.key_averages(group_by_stack_n=10).table(sort_by="cuda_memory_usage", row_limit=10))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
     

In [7]:
repeat, empty = sorted(prof.key_averages(group_by_stack_n=10), key=lambda e:e.cuda_memory_usage, reverse=True)[:2]

In [8]:
bert_tests.test_attention_fn(bert_attention)

attention MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: 0.002252 STD: 0.12 VALS [-0.09587 -0.2646 0.1785 0.04304 0.002363 0.04754 0.1104 0.03812 -0.09448 0.02245...]


In [17]:
fea.stack

[]

In [20]:
x1 = t.randn((10,4,5))
x2 = x1.repeat(1,1,1,6)
x1.stride(), x2.stride()

((20, 5, 1), (1200, 120, 30, 1))

In [19]:
x1.data_ptr(), x2.data_ptr()

(4808905216, 4831009984)